In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,33549
2,Huelva,Confirmados PDIA 14 días,864
3,Huelva,Tasa PDIA 14 días,"168,36525907593975"
4,Huelva,Confirmados PDIA 7 días,364
5,Huelva,Total Confirmados,33734
6,Huelva,Curados,31837
7,Huelva,Fallecidos,394


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  33549.0
/tmp/ipykernel_87705/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9182.0
/tmp/ipykernel_87705/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_87705/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_87705/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_87705/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1483 personas en los últimos 7 días 

Un positivo PCR cada 535 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,33549.0,364.0,864.0,513170.0,70.931660,168.365259,81.0
Huelva-Costa,19340.0,238.0,496.0,289548.0,82.197080,171.301477,47.0
Condado-Campiña,10904.0,102.0,307.0,156231.0,65.287939,196.503895,29.0
Huelva (capital),9182.0,97.0,269.0,143837.0,67.437447,187.017249,16.0
Moguer,1424.0,31.0,64.0,21867.0,141.766132,292.678465,10.0
Ayamonte,1370.0,31.0,38.0,21104.0,146.891585,180.060652,8.0
Bollullos Par del Condado,1045.0,13.0,22.0,14387.0,90.359352,152.915827,7.0
Isla Cristina,2203.0,20.0,31.0,21393.0,93.488524,144.907213,7.0
Sierra de Huelva-Andévalo Central,2951.0,23.0,58.0,67391.0,34.129186,86.064905,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),156.0,2.0,9.0,2327.0,85.947572,386.764074,0.0
San Juan del Puerto,593.0,10.0,35.0,9411.0,106.258634,371.905217,0.0
Paymogo,79.0,3.0,4.0,1162.0,258.175559,344.234079,1.0
Moguer,1424.0,31.0,64.0,21867.0,141.766132,292.678465,10.0
Palos de la Frontera,734.0,10.0,34.0,11742.0,85.164367,289.558849,3.0
Manzanilla,109.0,3.0,6.0,2118.0,141.643059,283.286119,0.0
Bonares,307.0,10.0,16.0,6060.0,165.016502,264.026403,3.0
Alosno,189.0,10.0,10.0,3933.0,254.258835,254.258835,1.0
Cartaya,1415.0,25.0,49.0,20083.0,124.483394,243.987452,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Trigueros,334.0,1.0,6.0,7862.0,12.719410,76.316459,0.0,0.166667
Palma del Condado (La),1199.0,4.0,22.0,10801.0,37.033608,203.684844,1.0,0.181818
Almonte,1921.0,8.0,43.0,24507.0,32.643734,175.460073,2.0,0.186047
Lucena del Puerto,268.0,1.0,5.0,3261.0,30.665440,153.327200,0.0,0.200000
Cerro de Andévalo (El),156.0,2.0,9.0,2327.0,85.947572,386.764074,0.0,0.222222
Valverde del Camino,669.0,7.0,26.0,12750.0,54.901961,203.921569,3.0,0.269231
San Juan del Puerto,593.0,10.0,35.0,9411.0,106.258634,371.905217,0.0,0.285714
Palos de la Frontera,734.0,10.0,34.0,11742.0,85.164367,289.558849,3.0,0.294118
Gibraleón,658.0,5.0,17.0,12737.0,39.255712,133.469420,3.0,0.294118
